In [1]:
import numpy as np
import tensorflow as tf

In [8]:
zeros = np.zeros((3, 2, 2, 1))

In [22]:
anchor = np.array([[[[1.], [1.]], [[1.], [1.]]], [[[1.], [1.]], [[1.], [1.]]]])
positive = np.array([[[[0.5], [1]], [[0.3], [1]]], [[[0.1], [0.1]], [[0.1], [0.1]]]])
negative = np.array([[[[1.], [1.]], [[1.], [1.]]], [[[1.], [1.]], [[1.], [1.]]]])

anchor.shape

(2, 2, 2, 1)

## Non-FLAT

In [23]:
subtract = np.subtract(anchor, positive)
print(subtract)

[[[[0.5]
   [0. ]]

  [[0.7]
   [0. ]]]


 [[[0.9]
   [0.9]]

  [[0.9]
   [0.9]]]]


In [24]:
square = np.square(subtract)
print(square)

[[[[0.25]
   [0.  ]]

  [[0.49]
   [0.  ]]]


 [[[0.81]
   [0.81]]

  [[0.81]
   [0.81]]]]


In [25]:
summed = np.sum(square, 1, keepdims=True)
print(summed)

[[[[0.74]
   [0.  ]]]


 [[[1.62]
   [1.62]]]]


In [26]:
positive_distance = np.sqrt(summed)
print(positive_distance)

[[[[0.86023253]
   [0.        ]]]


 [[[1.27279221]
   [1.27279221]]]]


In [27]:
np.mean(positive_distance)

0.8514542347439584

## FLAT

In [28]:
print(positive.reshape(-1, 4))
# (x1 - x2)^2 + (y1 - y2)^2 + (z1 - z2)^2 + ...

[[0.5 1.  0.3 1. ]
 [0.1 0.1 0.1 0.1]]


In [29]:
subtract = np.subtract(anchor.reshape(-1, 4), positive.reshape(-1, 4))
print(subtract)

[[0.5 0.  0.7 0. ]
 [0.9 0.9 0.9 0.9]]


In [30]:
square = np.square(subtract)
print(square)

[[0.25 0.   0.49 0.  ]
 [0.81 0.81 0.81 0.81]]


In [31]:
summed = np.sum(square, 1, keepdims=True)
print(summed)

[[0.74]
 [3.24]]


In [32]:
positive_distance = np.sqrt(summed)
print(positive_distance)

[[0.86023253]
 [1.8       ]]


In [33]:
np.mean(positive_distance)

1.3301162633521313

## Reshape Normalize

In [13]:
x = tf.constant([[3.0, 4.0], [3.0, 4.0]])
tf.math.l2_normalize(x, axis=1).numpy()

2022-05-12 15:37:49.876754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 15:37:49.879890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 15:37:49.880031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 15:37:49.880684: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

array([[0.6, 0.8],
       [0.6, 0.8]], dtype=float32)

In [14]:
w = tf.constant(positive.reshape(-1, 4))
wt = tf.math.l2_normalize(w, axis=1).numpy()
print(wt)

[[0.32686023 0.65372045 0.19611614 0.65372045]
 [0.5        0.5        0.5        0.5       ]]


## Calculate weight

In [7]:
no_offset_emb = np.array([[0.5, 0.5]])
offset_emb = np.array([[0.7, 0.7], [0.51, 0.51], [0.3, 0.3]])

max_distance = np.prod(no_offset_emb.shape)
print("MAX", max_distance)

all_distances = np.square(offset_emb - no_offset_emb)

for pair_distance in all_distances:
    distance = np.sqrt(np.nansum(pair_distance))

    normalized_distance = 1 - (distance / max_distance) # 1-similar, 0-different
    print(normalized_distance)

MAX 2
0.8585786437626906
0.9929289321881345
0.8585786437626904
